In [ ]:
import numpy as np
from matplotlib import pyplot

In [ ]:
def plot_packing(delta_a, delta_b):
    periodic_a = (np.arange(9) - 4) * delta_a
    periodic_b = (np.arange(9) - 4) * delta_b
    fig, ax = pyplot.subplots(figsize=(16, 32))
    pyplot.tight_layout()
    ax.set_aspect('equal')
    ax.set_xlim(-5.0, 5.0)
    ax.grid(True)
    ax.set_xticks(periodic_a)
    ax.set_yticks(periodic_b)
    ax.xaxis.set_major_formatter(pyplot.FormatStrFormatter('%.1f'))
    ax.yaxis.set_major_formatter(pyplot.FormatStrFormatter('%.1f'))
    for x_a in periodic_a:
        for x_b in periodic_b:
            ax.plot([x_a + 0.5, x_a - 0.5], [x_b - 0.5, x_b + 0.5], linewidth=2)
        ax.plot([])
    ax.add_patch(pyplot.Rectangle((-delta_a/2, -delta_b/2), delta_a, delta_b, linestyle='dashed', edgecolor='grey', facecolor='none'))

plot_packing(1.0, 1.0)
plot_packing(1.0, 0.2)

In [ ]:
periods = 9
period_res = 1001
plot_res = period_res * periods

def sample_positions(res, size):
    indices = np.arange(res)
    return ((indices + 0.5) / res - 0.5) * size

x = sample_positions(plot_res, periods)
periodic_pulse = sample_positions(periods, periods)
box = np.abs(x) < 0.5

def example(frequency):
    signal = np.cos(2.0 * np.pi * frequency * x)
    windowed_signal = signal * box

    periodic_signal = np.zeros(plot_res)
    for i in range(periods):
        periodic_signal += np.roll(windowed_signal, i * period_res)

    fig, axs = pyplot.subplots(3, 2, figsize=(16, 10))
    pyplot.tight_layout()

    fig.suptitle('Signal with frequency equal to {:.1f}'.format(frequency), fontsize=16)

    for ax in axs.flat:
        ax.grid(True)
        ax.set_aspect('equal')
        ax.set_xlim(-4.0, 4.0)
        ax.set_xticks(np.linspace(-4, 4, 9))
        ax.set_ylim(-1.2, 1.2)
        ax.set_yticks([-1.0, 0.0, 1.0])

    for ax in axs[:, 0]:
        ax.vlines([-0.5, 0.5], -2.0, 2.0, linestyle='dashed', linewidth=1, color='lightgrey', zorder=-1)

    for ax in axs[:, 1]:
        ax.axhline(0.5, linestyle='dashed', linewidth=1, color='lightgrey', zorder=-1)

    axs[0, 0].plot(x, signal)
    axs[1, 0].plot(x, windowed_signal)
    axs[2, 0].plot(x, periodic_signal)

    axs[0, 1].vlines([frequency, -frequency], 0.0, 0.5)
    axs[0, 1].axhline()

    axs[1, 1].plot(x, 0.5 * (np.sinc(x - frequency) + np.sinc(x + frequency)))

    axs[2, 1].vlines(periodic_pulse, 0.0, 0.5 * (np.sinc(periodic_pulse - frequency) + np.sinc(periodic_pulse + frequency)))
    axs[2, 1].scatter(periodic_pulse, 0.5 * (np.sinc(periodic_pulse - frequency) + np.sinc(periodic_pulse + frequency)), zorder=2)

fig, axs = pyplot.subplots(1, 2, figsize=(16, 100))
pyplot.tight_layout()

for ax in axs.flat:
    ax.grid(True)
    ax.set_aspect('equal')
    ax.set_xlim(-4.0, 4.0)
    ax.set_xticks(np.linspace(-4, 4, 9))
    ax.set_ylim(-1.2, 1.2)
    ax.set_yticks([-1.0, 0.0, 1.0])

axs[0].plot(x, np.abs(x) < 0.5)
axs[0].vlines(periodic_pulse, 0.0, 1.0, color='tab:orange')
axs[0].scatter(periodic_pulse, [1]*periods, color='tab:orange', zorder=2)

axs[1].plot(x, np.sinc(x))
axs[1].vlines(periodic_pulse, 0.0, 1.0, color='tab:orange')
axs[1].scatter(periodic_pulse, [1]*periods, color='tab:orange', zorder=2)

example(0.8)
example(1.0)
example(1.5)
example(2.0)

In [ ]:
def plot_sinc_lines(ax, omega):
    ax.axhline(omega[1], linestyle='dashed', color='tab:green')
    ax.axvline(omega[0], linestyle='dashed', color='tab:blue')
    ax.axline(omega, omega + [1, -1], linestyle='dashed', color='tab:red')
    ax.axline(omega, omega + [1, -0.5], linestyle='dashed', color='tab:purple')
    ax.scatter(omega[0], omega[1], color='tab:orange', zorder=3)

fig, ax = pyplot.subplots(figsize=(5, 10))
pyplot.tight_layout()
ax.set_xlabel(r'$\omega_a$', fontsize=14)
ax.set_ylabel(r'$\omega_b$', fontsize=14)
ax.set_aspect('equal')
ax.set_xticks([0])
ax.set_yticks([0])
ax.set_xlim(-1.0, 1.0)
ax.set_ylim(-1.0, 1.0)
plot_sinc_lines(ax, np.array([0.5, -0.7]))
plot_sinc_lines(ax, np.array([-0.5, 0.7]))
_ = ax.legend(['a', 'b', 'r', 'p'], loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14)

In [ ]:
z_r = -17.0
z_a = 17.0
z_b = 34.0

s_p = 5.0
s_r = 5.0

s_a = s_r * z_a / abs(z_r) + s_p
s_b = s_r * z_b / abs(z_r) + s_p

In [ ]:
window_a = np.array([-0.5, 0.5]) * s_a
window_b = np.array([-0.5, 0.5]) * s_b
window_r = np.array([-0.5, 0.5]) * s_r
window_p = np.array([-0.5, 0.5]) * s_p

far_ends = np.array([100, -100, -100, 100])

window_r_a = np.repeat(window_r, 2) * z_a / z_r + far_ends
window_r_b = np.repeat(window_r, 2) * z_b / z_r + far_ends

window_p_a = far_ends * z_a / z_r + np.repeat(window_p, 2)
window_p_b = far_ends * z_b / z_r + np.repeat(window_p, 2)

fig, ax = pyplot.subplots(figsize=(5, 10))
pyplot.tight_layout()
ax.set_xlabel(r'$x_a$', fontsize=14)
ax.set_ylabel(r'$x_b$', fontsize=14)
ax.set_aspect('equal')
ax.set_xticks([0])
ax.set_yticks([0])
ax.set_xlim(-10.0, 10.0)
ax.set_ylim(-10.0, 10.0)
ax.fill(np.repeat(window_a, 2), far_ends, alpha=0.6, color='tab:green', label='a')
ax.fill(far_ends, np.repeat(window_b, 2), alpha=0.6, color='tab:blue', label='b')
ax.fill(window_r_a, window_r_b, alpha=0.6, color='tab:red', label='r')
ax.fill(window_p_a, window_p_b, alpha=0.6, color='tab:purple', label='p')
_ = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14)

If the display window is bigger than the eye's field of view:
$ \large \begin{align*}
L \cdot \text{box}_a \cdot \text{box}_b \cdot \text{box}_r \cdot \text{box}_p &= L \cdot \text{box}_r \cdot \text{box}_p \\
\hat{L} * \text{sinc}_a * \text{sinc}_b * \text{sinc}_r * \text{sinc}_p &= \hat{L} * \text{sinc}_r * \text{sinc}_p \\
\end{align*} $

However, in reality we need to reconstruct the signal before it enters the pupil:
$ \large \begin{align*}
((L \cdot \text{box}_a \cdot \text{box}_b) * H_{ab}) \cdot \text{box}_r \cdot \text{box}_p &= \; ? \\
((\hat{L} * \text{sinc}_a * \text{sinc}_b) \cdot \hat{H}_{ab}) * \text{sinc}_r * \text{sinc}_p &= \; ? \\
\end{align*} $

If we could reconstruct first:
$ \large \begin{align*}
((L * H_{ab}) \cdot \text{box}_a \cdot \text{box}_b) \cdot \text{box}_r \cdot \text{box}_p &= (L * H_{ab}) \cdot \text{box}_r \cdot \text{box}_p \\
((\hat{L} \cdot \hat{H}_{ab}) * \text{sinc}_a * \text{sinc}_b) * \text{sinc}_r * \text{sinc}_p &= (\hat{L} \cdot \hat{H}_{ab}) * \text{sinc}_r * \text{sinc}_p \\
\end{align*} $

If we could reconstruct last:
$ \large \begin{align*}
((L \cdot \text{box}_a \cdot \text{box}_b) \cdot \text{box}_r \cdot \text{box}_p) * H_{ab} &= (L \cdot \text{box}_r \cdot \text{box}_p) * H_{ab} \\
((\hat{L} * \text{sinc}_a * \text{sinc}_b) * \text{sinc}_r * \text{sinc}_p) \cdot \hat{H}_{ab} &= (\hat{L} * \text{sinc}_r * \text{sinc}_p) \cdot \hat{H}_{ab} \\
\end{align*} $

In [ ]:
n = 9

x_a = np.tile((np.arange(2*n) - n + 0.5) * s_a/n, 2*n)
x_b = np.repeat((np.arange(2*n) - n + 0.5) * s_b/n, 2*n)

wa_x_a = np.tile((np.arange(n) - n/2 + 0.5) * s_a/n, n)
wa_x_b = np.repeat((np.arange(n) - n/2 + 0.5) * s_b/n, n)

mask_rp = np.logical_and(np.abs(wa_x_b - wa_x_a)*z_r/(z_a - z_b) < s_r/2, np.abs(wa_x_a * z_b - wa_x_b * z_a)/(z_b - z_a) < s_p/2)

corner_a = (s_r * z_a / z_r + s_p)/2
corner_b = (s_r * z_b / z_r + s_p)/2

fig, axs = pyplot.subplots(4, 3, figsize=(16, 21), sharex=True, sharey=True)
pyplot.tight_layout()

for ax in axs.flat:
    ax.set_xlabel(r'$x_a$', fontsize=14)
    ax.set_ylabel(r'$x_b$', fontsize=14)
    ax.set_aspect('equal')
    ax.set_xticks([0])
    ax.set_yticks([0])
    ax.set_xlim(-10.0, 10.0)
    ax.set_ylim(-10.0, 10.0)

for ax in axs[(2, 1, 1), (0, 1, 2)]:
    ax.fill(np.repeat(window_a, 2), far_ends, alpha=0.05, color='tab:green', label='a', zorder=5)
    ax.fill(far_ends, np.repeat(window_b, 2), alpha=0.05, color='tab:blue', label='b', zorder=5)

for ax in axs[(3, 3, 2), (0, 1, 2)]:
    ax.fill(window_r_a, window_r_b, alpha=0.05, color='tab:red', label='r', zorder=5)
    ax.fill(window_p_a, window_p_b, alpha=0.05, color='tab:purple', label='p', zorder=5)

for ax in axs[0, :]:
    ax.scatter(x_a, x_b, color='tab:orange', s=1.0, zorder=3)

axs[1, 0].fill(np.repeat(x_a, 5) + np.tile(np.array([-1, -1, 1, 1, np.nan]) * (s_a/n * 0.5 - 0.1), 4*n**2), np.repeat(x_b, 5) + np.tile(np.array([-1, 1, 1, -1, np.nan]) * (s_b/n * 0.5 - 0.1), 4*n**2), color='tab:orange')

for ax in axs[1, 1:3]:
    ax.scatter(wa_x_a, wa_x_b, color='tab:orange', s=1.0, zorder=3)

for ax in axs[2:4, 0:2].flat:
    ax.fill(np.repeat(wa_x_a, 5) + np.tile(np.array([-1, -1, 1, 1, np.nan]) * (s_a/n * 0.5 - 0.1), n**2), np.repeat(wa_x_b, 5) + np.tile(np.array([-1, 1, 1, -1, np.nan]) * (s_b/n * 0.5 - 0.1), n**2), color='tab:orange')

axs[2, 2].scatter(wa_x_a[mask_rp], wa_x_b[mask_rp], color='tab:orange', s=1.0, zorder=3)

axs[3, 2].fill(np.repeat(wa_x_a[mask_rp], 5) + np.tile(np.array([-1, -1, 1, 1, np.nan]) * (s_a/n * 0.5 - 0.1), 15), np.repeat(wa_x_b[mask_rp], 5) + np.tile(np.array([-1, 1, 1, -1, np.nan]) * (s_b/n * 0.5 - 0.1), 15), color='tab:orange')

for ax in axs[3, (0, 1)]:
    ax.fill([-10, s_a/2, -corner_a, -s_a/2], [10, s_b/2, -corner_b, -s_b/2], color='white')
    ax.fill([10, -s_a/2, corner_a, s_a/2], [-10, -s_b/2, corner_b, s_b/2], color='white')

In the spatial domain we have:
$ \large \begin{align*}
    n &= \text{number of spatial samples} \\
    s &= \text{size of the spatial sampling window} \\
    \Delta x &= \text{spatial sampling pitch} \\
\end{align*} $

In the frequency domain we have:
$ \large \begin{align*}
    n &= \text{number of frequency samples} \\
    \sigma &= \text{size of the frequency sampling window} \\
    \Delta \omega &= \text{frequency sampling pitch} \\
\end{align*} $

We can always describe $\Delta x$, $\Delta \omega$ and $\sigma$ in terms of $n$ and $s$:
$ \large \begin{align*}
    \Delta x &= \frac{s}{n} && (1)\\
    \Delta \omega &= \frac{1}{s} && (2)\\
    \sigma &= \frac{n}{s} && (3)\\
\end{align*} $

Display frequencies and eye frequencies relate to each other according to:
$ \large \begin{align*}
    z_{a} \, \omega_{a} + z_{b} \, \omega_{b} &= z_{r} \, \omega_{r} && (4)\\
    (z_{f} - z_{a}) \, \omega_{a} + (z_{f} - z_{b}) \, \omega_{b} &= z_{f} \, \omega_{p} && (5)\\
\end{align*} $

For a two-dimensional scene it is convenient to choose $z_{f} = \infty$. In addition to simplifying the equations it is also the most comfortable focus distance for the viewer.
$ \large \begin{align*}
    z_{f} = \infty \implies \omega_{a} + \omega_{b} = \omega_{p} && (6)\\
\end{align*} $

The $\omega_{r}$ and $\omega_{p}$ axes are defined as $\omega_{p} = 0$ and $\omega_{r} = 0$ respectively. Therefore they can be described in display space as:
$ \large \begin{align*}
    \omega_{p} = 0 &\implies
    \begin{cases}
        \omega_{a} = \frac{-z_{r}}{z_{b} - z_{a}} \omega_{r} \\
        \omega_{b} = \frac{-z_{r}}{z_{a} - z_{b}} \omega_{r} \\
    \end{cases} && (7)\\
    \omega_{r} = 0 &\implies
    \begin{cases}
        \omega_{a} = \frac{z_{b}}{z_{b} - z_{a}} \omega_{p} \\
        \omega_{b} = \frac{z_{a}}{z_{a} - z_{b}} \omega_{p} \\
    \end{cases} && (8)\\
\end{align*} $

To avoid interference among spectral replicas due to spectral leakage from the pupil we need:
$ \large \begin{align*}
    \sigma_{a} = \kappa_{a} \left\lvert \frac{z_{b}}{z_{b} - z_{a}} \right\rvert \Delta\omega_{p} && , \, \kappa_{a} \in \Z^{+} && (9)\\
    \sigma_{b} = \kappa_{b} \left\lvert \frac{z_{a}}{z_{a} - z_{b}} \right\rvert \Delta\omega_{p} && , \, \kappa_{b} \in \Z^{+} && (10)\\
\end{align*} $

Which in terms of $n$ and $s$ becomes:
$ \large \begin{align*}
    n_{a} = \kappa_{a} \left\lvert \frac{z_{b}}{z_{b} - z_{a}} \right\rvert \frac{s_{a}}{s_{p}} && (11)\\
    n_{b} = \kappa_{b} \left\lvert \frac{z_{a}}{z_{a} - z_{b}} \right\rvert \frac{s_{b}}{s_{p}} && (12)\\
\end{align*} $

To separate spectral replicas over the $\omega_{a}$ axis regardless of the choice of $\sigma_{b}$ (i.e. the naive method) we need:
$ \large \begin{align*}
    \sigma_{a} \geq \left\lvert \frac{z_{r}}{z_{b} - z_{a}} \right\rvert \sigma_{r} && (13)\\
\end{align*} $

Which in terms of $n$ and $s$ becomes:
$ \large \begin{align*}
    n_{a} \geq \left\lvert \frac{z_{r}}{z_{b} - z_{a}} \right\rvert \frac{s_{a}}{s_{r}} \, n_{r} && (14)\\
\end{align*} $

From $(11)$ and $(14)$ we have:
$ \large \begin{align*}
    \kappa_{a} \left\lvert \frac{z_{b}}{z_{b} - z_{a}} \right\rvert \frac{s_{a}}{s_{p}} \geq \left\lvert \frac{z_{r}}{z_{b} - z_{a}} \right\rvert \frac{s_{a}}{s_{r}} \, n_{r} &&
    \therefore && \frac{1}{s_{p}} \geq \left\lvert \frac{z_{r}}{z_{b}} \right\rvert \frac{1}{s_{r} \, \kappa_{a}} \, n_{r} && (15)\\
\end{align*} $

From $(12)$ and $(15)$ we have:
$ \large \begin{align*}
    n_{b} \geq \left\lvert \frac{z_{a}}{z_{a} - z_{b}} \right\rvert \left\lvert \frac{z_{r}}{z_{b}} \right\rvert \frac{s_{b} \, \kappa_{b}}{s_{r} \, \kappa_{a}} \, n_{r} && (16)\\
\end{align*} $

To keep the display window bigger than the eye's field of view we need:
$ \large \begin{align*}
    s_{a} \geq s_{p} + \left\lvert \frac{z_{a}}{z_{r}} \right\rvert s_{r} && (17)\\
    s_{b} \geq s_{p} + \left\lvert \frac{z_{b}}{z_{r}} \right\rvert s_{r} && (18)\\
\end{align*} $

Combining $(14)$ with $(17)$ and $(16)$ with $(18)$ we have:
$ \large \begin{align*}
    n_{a} &\geq \frac{n_{r}}{\lvert z_{b} - z_{a} \rvert} \left( \lvert z_{a} \rvert + \frac{s_{p}}{s_{r}} \lvert z_{r} \rvert \right) && (19)\\
    n_{b} &\geq \frac{n_{r}}{\lvert z_{b} - z_{a} \rvert} \left( \lvert z_{a} \rvert + \frac{s_{p}}{s_{r}} \lvert z_{r} \rvert \frac{\lvert z_{a} \rvert}{\lvert z_{b} \rvert} \right) \frac{\kappa_{b}}{\kappa_{a}} && (20)\\
\end{align*} $

<br>

Choosing $z_{f} = \infty$:

(3) &nbsp; $\omega_{a} + \omega_{b} = \omega_{p}$

<br>

To avoid spectral leakage we need:

(6) &nbsp; $\sigma_{a} = \kappa_{ar} \frac{-z_{r}}{z_{b} - z_{a}} \Delta\omega_{r} \;$ , $\; \kappa_{ar} \in \mathbb{Z}^{+}$

(7) &nbsp; $\sigma_{b} = \kappa_{br} \frac{-z_{r}}{z_{b} - z_{a}} \Delta\omega_{r} \;$ , $\; \kappa_{br} \in \mathbb{Z}^{+}$

(8) &nbsp; $\sigma_{a} = \kappa_{ap} \frac{z_{b}}{z_{b} - z_{a}} \Delta\omega_{p} \;$ , $\; \kappa_{ap} \in \mathbb{Z}^{+}$

(9) &nbsp; $\sigma_{b} = \kappa_{bp} \frac{z_{a}}{z_{b} - z_{a}} \Delta\omega_{p} \;$ , $\; \kappa_{bp} \in \mathbb{Z}^{+}$

<br>

Therefore:

(10) &nbsp; $z_{a} = -z_{r} \frac{\kappa_{br}}{\kappa_{bp}} \frac{\Delta \omega_{r}}{\Delta \omega_{p}} = -z_{r} \frac{\kappa_{br}}{\kappa_{bp}} \frac{s_{p}}{s_{r}}$

(11) &nbsp; $z_{b} = -z_{r} \frac{\kappa_{ar}}{\kappa_{ap}} \frac{\Delta \omega_{r}}{\Delta \omega_{p}} = -z_{r} \frac{\kappa_{ar}}{\kappa_{ap}} \frac{s_{p}}{s_{r}}$

<br>

Choosing to space spectral replicas over the $\omega_{a}$ axis according to the naive method:

(12) &nbsp; $\sigma_{a} \geq \frac{-z_{r}}{z_{b} - z_{a}} \sigma_{r} = n_{r} \frac{-z_{r}}{z_{b} - z_{a}} \Delta\omega_{r}$

<br>

Choosing $\kappa_{ar} = n_{r}$:

(13) &nbsp; $z_{b} = -z_{r} \frac{n_{r}}{\kappa_{ap}} \frac{s_{p}}{s_{r}}$

<br>

Choosing $\kappa_{bp} = 1$:

(14) &nbsp; $z_{a} = -z_{r} \, \kappa_{br} \frac{s_{p}}{s_{r}}$

<br>

Keep in mind that:

(15) &nbsp; $z_{b} > z_{a} \implies n_{r} > \kappa_{ap} \, \kappa_{br}$

(16) &nbsp; $z_{b} - z_{a} = -z_{r} (\frac{n_{r}}{\kappa_{ap}} - \kappa_{br}) \frac{s_{p}}{s_{r}}$

(17) &nbsp; $\frac{-z_{r}}{z_{b} - z_{a}} = \frac{\kappa_{ap}}{n_{r} - \kappa_{ap} \, \kappa_{br}} \frac{s_{r}}{s_{p}}$

<br>

To keep the display window bigger than the eye's field of view, we need:

(18) &nbsp; $s_{a} \geq \frac{z_{a}}{-z_{r}} s_{r} + s_{p} = (\kappa_{br} + 1) \, s_{p}$

(19) &nbsp; $s_{b} \geq \frac{z_{b}}{-z_{r}} s_{r} + s_{p} = (\frac{n_{r}}{\kappa_{ap}} + 1) \, s_{p}$

<br>

The display resolution is then restricted to:

(20) &nbsp; $n_{a} = s_{a} \, \sigma_{a} \geq (\kappa_{br} + 1) \, s_{p} \, n_{r} \frac{-z_{r}}{z_{b} - z_{a}} \Delta\omega_{r} = (\kappa_{br} + 1) \, n_{r} \frac{\kappa_{ap}}{n_{r} - \kappa_{ap} \, \kappa_{br}}$
$= \frac{n_{r} \, \kappa_{ap} \, \kappa_{br} + n_{r} \, \kappa_{ap}}{n_{r} - \kappa_{ap} \, \kappa_{br}}$

(21) &nbsp; $n_{b} = s_{b} \, \sigma_{b} \geq (\frac{n_{r}}{\kappa_{ap}} + 1) \, s_{p} \, \kappa_{br} \frac{-z_{r}}{z_{b} - z_{a}} \Delta\omega_{r} = (\frac{n_{r}}{\kappa_{ap}} + 1) \, \kappa_{br} \frac{\kappa_{ap}}{n_{r} - \kappa_{ap} \, \kappa_{br}}$
$= \frac{n_{r} \, \kappa_{br} + \kappa_{ap} \, \kappa_{br}}{n_{r} - \kappa_{ap} \, \kappa_{br}}$

<br>

Choosing $\kappa_{br} = 1$:

(22) &nbsp; $n_{a} \geq 2 \frac{n_{r} \, \kappa_{ap}}{n_{r} - \kappa_{ap}}$

(23) &nbsp; $n_{b} \geq \frac{n_{r} + \kappa_{ap}}{n_{r} - \kappa_{ap}}$

(24) &nbsp; $z_{a} = -z_{r} \frac{s_{p}}{s_{r}}$

<br>

Let $\kappa_{ap} = n_{r} - \eta$ , $\eta \in \mathbb{Z} \cap [1, \, n_{r} - 1]$:

(25) &nbsp; $n_{a} \geq 2 \frac{n_{r} \, (n_{r} - \eta)}{\eta} = 2 (\frac{n_{r}^{2}}{\eta} - n_{r})$

(26) &nbsp; $n_{b} \geq \frac{n_{r} + (n_{r} - \eta)}{\eta} = 2 \frac{n_{r}}{\eta} - 1$

(27) &nbsp; $z_{b} = -z_{r} \frac{n_{r}}{n_{r} - \eta} \frac{s_{p}}{s_{r}} = \frac{n_{r}}{n_{r} - \eta} z_{a}$

<br>

For $n_{r} = 256$ and $\eta = 1$:

$n_{a} \geq 130560$

$n_{b} \geq 511$

$z_{b} = 1.003921568627451 \, z_{a}$

<br>

For $n_{r} = 256$ and $\eta = 128$:

$n_{a} \geq 512$

$n_{b} \geq 3$

$z_{b} = 2 \, z_{a}$

<br>

For $n_{r} = 256$ and $\eta = 255$:

$n_{a} \geq 2.007843137254895$

$n_{b} \geq 1.007843137254902$

$z_{b} = 256 \, z_{a}$

In [ ]:
n_r = 256
n_a = 512
n_b = 3

pass